#### 通过使用深度学习框架来简洁实现  线性回归模型  生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=d2l.synthetic_data(true_w,true_b,1000)


#### 调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器。"""
    # 将特征和标签作为一个list传入tensor的dataset中
    dataset = data.TensorDataset(*data_arrays)
    # 调用dataloader函数，每次随机挑选b个样本出来
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

# 得到X和y
next(iter(data_iter))

[tensor([[-0.4939, -0.8643],
         [-1.3224,  2.4071],
         [ 0.9919,  0.1928],
         [ 0.8697,  0.9612],
         [ 0.8681,  0.0336],
         [-0.1123, -0.1200],
         [-0.1311, -0.1418],
         [ 2.0302, -1.3842],
         [ 0.4837,  0.1921],
         [-0.0527,  1.3166]]),
 tensor([[ 6.1482],
         [-6.6195],
         [ 5.5190],
         [ 2.6510],
         [ 5.8363],
         [ 4.3633],
         [ 4.3978],
         [12.9664],
         [ 4.5064],
         [-0.3834]])]

#### 使用框架预先定义好的层

In [3]:
from torch import nn
# list of layers = Sequentials按顺序排列好的层
net = nn.Sequential(nn.Linear(2, 1)) # 输入维度2，输出维度1

#### 初始化模型参数

In [5]:
# net可以通过下标0访问到第一层（线性回归层）
net[0].weight.data.normal_(0,0.01) # 使用正态分布来替换weight的值
net[0].bias.data.fill_(0)

tensor([0.])

#### 计算均方误差使用的是MSELoss类，平方 $L_2$ 范数

In [6]:
loss=nn.MSELoss()

#### 实例化SGD实例

In [7]:
trainer=torch.optim.SGD(net.parameters(),lr=0.03)

#### 训练代码与之前类似

In [20]:
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter:
        # 拿出层里面的数据计算loss
        l = loss(net(X),y)
        # 梯度清零
        trainer.zero_grad()
        # 计算梯度，自动求sum，不需要自己去sum
        l.backward()
        # 模型更新
        trainer.step()
    # 扫完一遍数据之后，将所有features放到net中，跟所有的labels做一次loss并且打印比较结果
    l = loss(net(features),labels)
    print(f'epoch{epoch+1},loss{l:f}')

epoch1,loss0.000109
epoch2,loss0.000109
epoch3,loss0.000109
